In [1]:
"""
import pandas as pd
import numpy as np
import csv
import findspark
findspark.init('/home/hadoop/spark-2.2.2-bin-hadoop2.7')
import pyspark
sc = pyspark.SparkContext()
from pyspark.sql import SQLContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
spark = SparkSession(sc)
sqlContext = SQLContext(sc)
"""

import findspark
import pandas as pd
import numpy as np
import csv
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk1.8.0_241"
os.environ["SPARK_HOME"] = "/opt/spark2.2.2"
os.environ["PYSPARK_PYTHON"] = "/home/aaa/anaconda3/envs/py356/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/home/aaa/anaconda3/envs/py356/bin/python"
findspark.init()
memory = '20g'
pyspark_submit_args = '--jars /home/aaa/scala/target/scala-2.11/cosine-similarity_2.11-1.0.jar --master local[24] --driver-memory ' + memory + ' --executor-cores 2 --num-executors 12 --executor-memory 20g --conf spark.ui.port=54353 pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *

SparkContext.setSystemProperty('spark.driver.memory', '20g')
SparkContext.setSystemProperty('spark.executor.memory', '20g')
sc = SparkContext('local[24]')
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [2]:
import csv
import ast
import pyspark.sql.functions as F
"""
wadiz_schema = StructType([
    StructField("idx", IntegerType()),
    StructField("Unnamed: 0", IntegerType()),
    StructField("Unnamed: 0.1", IntegerType()),
    StructField("Unnamed: 0.1.1", IntegerType()),
    StructField("id", IntegerType()),
    StructField("name", StringType()),
    StructField("category", StringType()),
    StructField("makerName", StringType()),
    StructField("summary", StringType()),
    StructField("achievementRate", IntegerType()),
    StructField("totalAmount", IntegerType()),
    StructField("totalSupporter", IntegerType()),
    StructField("totalLike", IntegerType()),
    StructField("rewardSatisfaction", DoubleType()),
    StructField("makerSatisfaction", DoubleType()),
    StructField("detailUrl", StringType()),
    StructField("campaigncomments", StringType()),
    StructField("comments", StringType())
])

spark_df =  spark.read.option("multiline",'true').csv("file:///home/aaa/wadiz/wadiz_final.csv", header=True, inferSchema=True, schema=wadiz_schema)
spark_df_droped = spark_df.select("id", "name", "category", "makerName", "summary", "achievementRate", "totalAmount", "totalSupporter", "totalLike")
spark_df_droped.printSchema()

spark_df_droped.write.csv(path="file:///home/aaa/wadiz/wadiz_final1.csv", header='true', mode='overwrite')
"""

# csv 파일 한도 늘리기 && pandas > spark dataframe 데이터 깨짐
csv.field_size_limit(1000000)
fr = open('rdd/wadiz_final.csv', 'r', encoding='utf-8')
fw = open('rdd/wadiz_final2.csv', 'w', encoding='utf-8')
rdr = csv.reader(fr)
wadiz_list = []
wadiz_list.append(['id', 'name', 'category', 'makerName', 'summary', 'achievementRate', 
                   'totalAmount', 'totalSupporter','totalLike'])
count = 0
for i,line in enumerate(rdr):
    if(i == 0):
        continue
    id = line[4]
    name = line[5]
    category = line[6]
    makerName = line[7]
    summary = line[8].replace('\n','')
    achievementRate = line[9]
    totalAmount = line[10]
    totalSupporter = line[11]
    totalLike = line[12]
    wadiz_list.append([id, name, category, makerName, summary, achievementRate, totalAmount, 
                       totalSupporter, totalLike])
    
wtr = csv.writer(fw)
for row in wadiz_list:
    wtr.writerow(row)
print(len(wadiz_list))
fr.close()
fw.close()


16783


In [187]:
%%time
from io import StringIO

csv.field_size_limit(1000000)
def loadRecord(line):
    csv.field_size_limit(1000000)
    input = StringIO(line)
    reader = csv.DictReader(input, fieldnames=["id", "name", "category", "makerName", "summary", "achievementRate", "totalAmount", "totalSupporter", "totalLike"])
    
    return next(reader)

spark_rdd = sc.textFile("file:///home/aaa/wadiz/rdd/wadiz_final2.csv").map(loadRecord)
header = spark_rdd.first()
spark_rdd = spark_rdd.filter(lambda line : line != header)
spark_rdd.collect()

CPU times: user 52.4 ms, sys: 8.18 ms, total: 60.6 ms
Wall time: 263 ms


In [188]:
%%time
def make_type(line):
    if (line['summary'] is None):
        line['summary'] = '0'
    else:
        line['summary'] = line['summary'].replace('\n','')
    
    if (line['category'] is None):
        line['category'] = '0'
    
    if (line['makerName'] is None):
        line['makerName'] = '0'
    line['id'] = int(line['id'])
    line['achievementRate'] = int(line['achievementRate'])
    line['totalAmount'] = int(line['totalAmount'])
    line['totalSupporter'] = int(line['totalSupporter'])
    line['totalLike'] = int(line['totalLike'])
    line['soop'] = (line['category']) * 1 + (line['makerName'] + ' ' )*2 + line['summary']
    line['soop'] = line['soop'].replace('\n','').replace('\0','')
    return line
 
spark_rdd = spark_rdd.map(make_type)
spark_rdd.collect()

CPU times: user 56 ms, sys: 8.16 ms, total: 64.1 ms
Wall time: 268 ms


In [189]:
%%time
def divide(line):
    totalSupporter = int(line['totalSupporter'])
    totalAmount = int(line['totalAmount'])
    if(totalSupporter is None or totalAmount is None):
        line['price'] = 0
        line['rangeAmount'] = 0
    elif (totalSupporter == 0 or totalAmount == 0):
        line['price'] = 0
        line['rangeAmount'] = 0
    else: 
        price = int(totalAmount / totalSupporter)
        if 0 <= price and price <30000:
            line['rangeAmount'] = 0
        elif 30000<= price and price <50000:
            line['rangeAmount'] = 1
        elif 50000<= price and price <70000:
            line['rangeAmount'] = 2
        elif 70000<= price and price <100000:
            line['rangeAmount'] = 3
        elif 100000<= price and price <200000:
            line['rangeAmount'] = 4
        elif 200000<= price and price< 300000:
            line['rangeAmount'] = 5
        elif 300000<= price and price <400000:
            line['rangeAmount'] = 6
        elif 400000<= price and price <500000:
            line['rangeAmount'] = 7
        else:
            line['rangeAmount'] = 8
        price = int(totalAmount / totalSupporter)
        line['price'] = price
    return line
    
    
spark_rdd = spark_rdd.map(divide)
spark_rdd.collect()

CPU times: user 67.2 ms, sys: 3.95 ms, total: 71.1 ms
Wall time: 286 ms


### Text Processing and Featurization

In [193]:
%%time
from pyspark.ml.feature import Tokenizer,HashingTF, Word2Vec
wadiz_schema = StructType([
    StructField("id", IntegerType()),
    StructField("name", StringType()),
    StructField("category", StringType()),
    StructField("makerName", StringType()),
    StructField("summary", StringType()),
    StructField("achievementRate", IntegerType()),
    StructField("totalAmount", IntegerType()),
    StructField("totalSupporter", IntegerType()),
    StructField("totalLike", IntegerType()),
    StructField("rewardSatisfaction", DoubleType()),
    StructField("makerSatisfaction", DoubleType()),
    StructField("rangeAmount", IntegerType()),
    StructField("soop", StringType())
])
spark_df = spark_rdd.toDF()
tokenizer = Tokenizer(inputCol='soop', outputCol='keywords')
wordData = tokenizer.transform(spark_df)
word2Vec = Word2Vec(vectorSize=100, minCount=5, inputCol='keywords', outputCol='word_vec', seed=123)
word2VecData = word2Vec.fit(wordData)
word2VecData = word2VecData.transform(wordData)
word2VecData_rdd = word2VecData.rdd


/opt/spark2.2.2/python/pyspark/sql/session.py:356: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


CPU times: user 14.5 ms, sys: 7.89 ms, total: 22.4 ms
Wall time: 3.5 s


In [87]:
word2VecData.printSchema()
spark_df.printSchema()
word2VecData_rdd.collect()

root
 |-- achievementRate: long (nullable = true)
 |-- category: string (nullable = true)
 |-- id: long (nullable = true)
 |-- makerName: string (nullable = true)
 |-- name: string (nullable = true)
 |-- price: long (nullable = true)
 |-- rangeAmount: long (nullable = true)
 |-- soop: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- totalAmount: long (nullable = true)
 |-- totalLike: long (nullable = true)
 |-- totalSupporter: long (nullable = true)
 |-- keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- word_vec: vector (nullable = true)

root
 |-- achievementRate: long (nullable = true)
 |-- category: string (nullable = true)
 |-- id: long (nullable = true)
 |-- makerName: string (nullable = true)
 |-- name: string (nullable = true)
 |-- price: long (nullable = true)
 |-- rangeAmount: long (nullable = true)
 |-- soop: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- totalAmount: long (nullable = true)
 |-- to

[Row(achievementRate=7906, category='뷰티', id=65544, makerName='심플리웍스', name='[1.8억/7차] 묵은 피지와 각질도 끝장보는 클렌저 <세수한번 굿모링>', price=68159, rangeAmount=2, soop='뷰티심플리웍스 심플리웍스     진짜가 돌아왔다! 더 강력해진 <세수한번 굿모링 클렌저>가 블랙헤드, 각질은 물론 피부결 개선까지 책임질게요. 올 여름 피부, 세수한번만 믿고 따라오세요!       ', summary='    진짜가 돌아왔다! 더 강력해진 <세수한번 굿모링 클렌저>가 블랙헤드, 각질은 물론 피부결 개선까지 책임질게요. 올 여름 피부, 세수한번만 믿고 따라오세요!       ', totalAmount=39532400, totalLike=426, totalSupporter=580, keywords=['뷰티심플리웍스', '심플리웍스', '', '', '', '', '진짜가', '돌아왔다!', '더', '강력해진', '<세수한번', '굿모링', '클렌저>가', '블랙헤드,', '각질은', '물론', '피부결', '개선까지', '책임질게요.', '올', '여름', '피부,', '세수한번만', '믿고', '따라오세요!'], word_vec=DenseVector([0.0388, -0.0137, -0.0181, 0.0069, 0.0312, 0.0087, -0.0323, -0.0462, -0.0575, -0.0079, -0.0163, 0.0102, -0.007, -0.0428, -0.01, -0.0147, 0.0169, 0.0276, -0.0046, 0.0462, 0.0268, -0.0064, -0.0312, -0.0466, -0.0265, -0.0229, -0.0167, -0.0146, -0.0376, 0.03, -0.0021, 0.0215, -0.0131, -0.0116, -0.0396, 0.0421, -0.0168, -0.0169, -0.0412, 0.0047, -0.0526, -0

In [194]:
# rangeAmount 비교값 추가
all_wadiz_vecs = word2VecData_rdd.map(lambda x: (x[2], x[13], x[6])).collect()
#all_wadiz_vecs = word2VecData.select('id','word_vec','rangeAmount')

In [195]:
print(all_wadiz_vecs[1])
print(all_wadiz_vecs[1][0])
print(all_wadiz_vecs[1][1])
print(all_wadiz_vecs[1][2])

(9, DenseVector([0.0173, -0.0099, -0.014, -0.0012, 0.0221, 0.0134, -0.0266, -0.043, -0.0388, 0.0029, -0.0049, -0.0032, -0.0044, -0.0257, -0.0132, -0.0149, 0.0121, 0.0178, -0.004, 0.0348, 0.0208, -0.0058, -0.015, -0.0281, -0.0189, -0.0164, -0.0117, -0.0091, -0.0203, 0.0261, 0.0166, 0.0181, -0.0088, -0.0223, -0.0258, 0.029, -0.0143, -0.0114, -0.0255, 0.0068, -0.038, -0.0349, 0.0111, -0.004, 0.0086, -0.0002, -0.0028, 0.0058, 0.0053, 0.0243, 0.0019, 0.0113, 0.025, 0.0299, 0.0193, -0.0435, -0.0211, -0.0271, 0.0017, -0.05, 0.0227, 0.0004, -0.0079, 0.034, -0.0049, -0.0264, 0.0122, 0.0019, 0.0012, 0.0126, -0.0351, 0.0157, 0.0119, -0.0059, -0.0186, -0.018, 0.0108, 0.0031, 0.0092, -0.0252, 0.0023, -0.0052, -0.0328, -0.0329, -0.0075, -0.0044, 0.0036, 0.0225, 0.0225, 0.0209, -0.0265, -0.0076, -0.0473, 0.0081, 0.029, 0.0237, -0.0283, -0.006, -0.0063, 0.015]), 0)
9
[0.017314390927952315,-0.009919439001740622,-0.01404406050486224,-0.0012209172065403046,0.02208133677153715,0.013395204990436987,-0.0265

In [196]:
def CosineSim(vec1, vec2): 
    return vec1.dot(vec2)/(vec1.norm(2)*vec2.norm(2))
    #return np.dot(vec1, vec2) / np.sqrt(np.dot(vec1, vec1)) / np.sqrt(np.dot(vec2, vec2))

In [197]:
import pyspark.sql.functions as F
from pyspark.ml.feature import Normalizer
from pyspark.sql.column import Column, _to_java_column, _to_seq 

def cosinesimilarity_udf(a, b): 
    cosinesimilarityUDF = spark._jvm.cosinesimilarityUDFs.cosinesimilarityUDF() 
    return Column(cosinesimilarityUDF.apply(_to_seq(spark.sparkContext, [a, b], _to_java_column)))

# input: 상품 id 리스트 > output: 추천 상품 (input_id(입력한 상품 id), id(입력한 상품과 코사인 유사도가 제일 높은 상품 id), score(코사인 유사도 계산)))
def getSimilarProduct(w_ids, sim_product_limit=10):
    print(wids, type(wids))
    row_list = []
    for w_id in wids:
        w_id = int(w_id)
        input_vec = []
        for r in all_wadiz_vecs:
            if r[0] == w_id:
                input_vec = [(r[1])][0]
                input_rangeAmount = r[2]
            else:
                continue    

            for i in all_wadiz_vecs: 
                if input_rangeAmount - 2 < i[2] and input_rangeAmount + 2 > i[2] and w_id != i[0]:
                    score = float(CosineSim(input_vec, i[1]))
                    if score <= 1:
                        row_list.append({'id' : i[0], 'score': score, 'input_id' : w_id})

    similar_product_rdd = sc.parallelize(row_list)          
    similar_product_rdd = similar_product_rdd.sortBy(lambda x: x['score'], False)
    return similar_product_rdd


In [198]:
# input: 추천 상품 (input_id, id, score) > output: inner join (input_id, id, score, name, category, makerName, summary)
def getProductDetails(in_product):
    wids = [54968, 53536, 42496, 30763, 34841]
    in_product = getSimilarProduct(wids)
    spark_rdd_map = spark_rdd.map(lambda x: (x['id'], (x['name'], x['category'], x['makerName'], x['summary'], x['rangeAmount'], x['totalAmount'])))
    in_product_map = in_product.map(lambda x: (x['id'], (x['score'])))

    joined_map = spark_rdd_map.join(in_product_map)

    joined_map = joined_map.map(lambda x: [x[0], x[1][0][0] , x[1][0][3], x[1][0][1], x[1][0][2],  x[1][0][5], x[1][1]]).sortBy(lambda x: x[6], False)
    return joined_map

### Test Recommend

In [199]:
%%time
import numpy as np
from pyspark.sql.functions import *
import math

all_wadiz_vecs = word2VecData_rdd.map(lambda x: (x[2], x[13], x[6])).collect()

# 상품 id 리스트
wids = [54968, 53536, 42496, 30763, 34841]

# 상품 추천 및 추천 상품 디테일 
sims = getProductDetails(getSimilarProduct(wids))
sims.toDF().limit(8).toPandas()


[54968, 53536, 42496, 30763, 34841] <class 'list'>


/home/aaa/anaconda3/envs/py356/lib/python3.5/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


[54968, 53536, 42496, 30763, 34841] <class 'list'>
CPU times: user 1.98 s, sys: 158 ms, total: 2.13 s
Wall time: 3.63 s


### User based Recomment

In [22]:
user_df =  spark.read.csv("file:///home/aaa/wadiz/users_merge_result2.csv", header=True, inferSchema=True)

In [23]:
user_df.select('user_id' ,'funding_id').show()
user_df.printSchema()

+---------+----------+
|  user_id|funding_id|
+---------+----------+
| 96394201|     65544|
| 96394201|     29175|
| 96394201|     43153|
|298659501|     65544|
|298659501|     68610|
|298659501|     59790|
|107362301|     65544|
|107362301|     68436|
|107362301|     30971|
|107362301|     63273|
|107362301|     54432|
|107362301|     62812|
|107362301|     59605|
|107362301|     57895|
|107362301|     57078|
|107362301|     51684|
|107362301|     54086|
|107362301|     51419|
|107362301|     48746|
|107362301|     46396|
+---------+----------+
only showing top 20 rows

root
 |-- user_id: integer (nullable = true)
 |-- funding_id: integer (nullable = true)



In [24]:
from pyspark.sql import Row
user_rdd = user_df.select('*').groupby('user_id').agg(F.collect_list('funding_id').alias('funding_ids')).rdd.collect()

In [26]:
# user_df = user_rdd.toDf()
for i, r in enumerate(user_rdd):
    u_id = r[0]
    funding_ids = list(r[1])
    print(u_id, funding_ids)
    sims = getProductDetails(getSimilarProduct(list(set(funding_ids))))
    sims = sims.withColumn('user_id', lit(u_id))
    sims = sims.select('user_id','id','name','category', 'amount','score').orderBy('score').limit(10)
    if(i == 0):
        allSims = sims
    else:
        allSims = allSims.union(sims)
        
    if(i == len(user_rdd)):
        allSims.to_json('./cbf_recommend.json', orient='records', lines=True)        


29601 [9, 9, 13, 275, 642, 642, 689, 733, 750, 750, 9265, 44833]
47501 [31]
57201 [31]
206501 [55]
227501 [66]
291501 [69, 69]
300601 [63, 58743]
312801 [49]
317201 [49, 9271, 16853]
324601 [1094, 42262]
353501 [87]
400301 [87]
441201 [142, 65614, 52236, 58339, 58274]
450801 [142]
460601 [136]
529301 [228]
570901 [177, 177, 62607, 62607]
664901 [195]
688301 [183]
701901 [239, 239, 457]
704301 [55978]
849401 [253]
867601 [253]
883701 [1553, 1553]
888901 [285, 890]
909801 [185]
928901 [317]
953701 [365]
955001 [340, 554, 755]
982701 [405]
1034201 [366]
1062801 [366]
1068901 [416]
1078201 [430]


KeyboardInterrupt: 